# Hello Tensorflow Too

## test 0: Fashion MNIST classification with tf.data.Datasets & tf.keras

here we create a small 2D convolutional network to classify MNIST digits.

it consists of a number of 2D convolutional layers that compress the 28x28 pixel digits to 64-dim vector before a final dense network for classification.

we can then test using this classification network as the encoder of an autoencoder model (next notebook).


In [1]:
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0-beta1'

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
# add axis for channel and normalize to 0 ~ 1 (based on training max)
x_test = np.expand_dims(x_test, axis=-1)/np.max(x_train)
x_train = np.expand_dims(x_train, axis=-1)/np.max(x_train)
x_shape = x_train[0].shape
print(np.min(x_train), np.max(x_train))
print(x_train.shape)
# add axis for sparse categorical crossentropy (unnecessary)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)
print(y_train.shape)

0.0 1.0
(60000, 28, 28, 1)
(60000, 1)


In [4]:
trn_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
trn_dataset = trn_dataset.shuffle(buffer_size=128).batch(32, drop_remainder=True)

In [5]:
tst_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
tst_dataset = tst_dataset.shuffle(buffer_size=128).batch(32, drop_remainder=True)

In [6]:
clfmdl = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=8, activation='relu', strides=1, input_shape=x_shape, name='conv2d_a'),
    tf.keras.layers.Dropout(rate=0.25, name='dropout_a'),
    tf.keras.layers.Conv2D(24, kernel_size=7, activation='relu', strides=1, input_shape=x_shape, name='conv2d_b'),
    tf.keras.layers.Conv2D(16, kernel_size=5, activation='relu', strides=1, input_shape=x_shape, name='conv2d_c'),
    tf.keras.layers.Dropout(rate=0.25, name='dropout_b'),
    tf.keras.layers.Conv2D(12, kernel_size=4, activation='relu', strides=1, kernel_regularizer=tf.keras.regularizers.l2(0.005), name='conv2d_d'),
    tf.keras.layers.Conv2D(8, kernel_size=3, activation='relu', strides=1, kernel_regularizer=tf.keras.regularizers.l2(0.005), name='conv2d_e'),
    tf.keras.layers.Dropout(rate=0.25, name='dropout_c'),
    tf.keras.layers.Conv2D(8, kernel_size=2, activation='relu', strides=1, kernel_regularizer=tf.keras.regularizers.l2(0.005), name='conv2d_f'),
    tf.keras.layers.Conv2D(4, kernel_size=2, activation='tanh', name='conv2d_g'),
    tf.keras.layers.Flatten(name='flatten'),
    tf.keras.layers.Dense(64, activation='relu', name='dense'),
    tf.keras.layers.Dense(10, activation='softmax', name='prediction')
], name='classifier')

In [7]:
clfmdl.summary()

Model: "classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_a (Conv2D)            (None, 21, 21, 32)        2080      
_________________________________________________________________
dropout_a (Dropout)          (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_b (Conv2D)            (None, 15, 15, 24)        37656     
_________________________________________________________________
conv2d_c (Conv2D)            (None, 11, 11, 16)        9616      
_________________________________________________________________
dropout_b (Dropout)          (None, 11, 11, 16)        0         
_________________________________________________________________
conv2d_d (Conv2D)            (None, 8, 8, 12)          3084      
_________________________________________________________________
conv2d_e (Conv2D)            (None, 6, 6, 8)           8

In [8]:
clfmdl.compile(optimizer=tf.keras.optimizers.Adam(),
               loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [9]:
clfmdl.fit(trn_dataset, epochs=20, verbose=2)

W0621 11:05:57.589309 139696230758144 deprecation.py:323] From /home/derek/miniconda3/envs/tf2beta/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
1875/1875 - 20s - loss: 0.8629 - sparse_categorical_accuracy: 0.6880
Epoch 2/20
1875/1875 - 19s - loss: 0.5613 - sparse_categorical_accuracy: 0.8021
Epoch 3/20
1875/1875 - 18s - loss: 0.4926 - sparse_categorical_accuracy: 0.8278
Epoch 4/20
1875/1875 - 18s - loss: 0.4540 - sparse_categorical_accuracy: 0.8406
Epoch 5/20
1875/1875 - 19s - loss: 0.4249 - sparse_categorical_accuracy: 0.8506
Epoch 6/20
1875/1875 - 19s - loss: 0.4051 - sparse_categorical_accuracy: 0.8585
Epoch 7/20
1875/1875 - 18s - loss: 0.3922 - sparse_categorical_accuracy: 0.8618
Epoch 8/20
1875/1875 - 18s - loss: 0.3790 - sparse_categorical_accuracy: 0.8656
Epoch 9/20
1875/1875 - 19s - loss: 0.3670 - sparse_categorical_accuracy: 0.8710
Epoch 10/20
1875/1875 - 18s - loss: 0.3609 - sparse_categorical_accuracy: 0.8719
Epoch 11/20
1875/1875 - 19s - loss: 0.3544 - sparse_categorical_accuracy: 0.8748
Epoch 12/20
1875/1875 - 18s - loss: 0.3478 - sparse_categorical_accuracy: 0.8764
Epoch 13/20
1875/1875 - 18s - loss: 0

In [10]:
clfmdl.evaluate(tst_dataset)

312/312 [==============================] - 1s 5ms/step - loss: 0.3620 - sparse_categorical_accuracy: 0.8758


[0.3620129776282761, 0.87580127]

## results

after twenty epochs, this network reaches 87.5% accuracy on test set